In [1]:
#All the imports here
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.objects.conversion.process_tree import converter as pt_converter
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.objects.conversion.dfg import converter as dfg_mining
from pm4py.visualization.petrinet import factory as pn_vis_factory
from collections import defaultdict 
import pandas as pd
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.adapters.pandas import csv_import_adapter
from pm4py.objects.conversion.log import factory as conversion_factory
from pm4py.util import constants
import math
from datetime import date
import datetime
import numpy as np
from pm4py.objects.log.util import sorting
from pm4py.algo.filtering.log.attributes import attributes_filter
from pm4py.statistics.traces.log import case_statistics

In [2]:
log_csv4 = pd.read_csv('Production_Data.csv', sep=',')
log_csv4.rename(columns={'Activity': 'concept:name','Start Timestamp':'time:timestamp'}, inplace=True)
log_csv4['time:timestamp'] =  pd.to_datetime(log_csv4['time:timestamp'] , format='%m/%d/%Y %H:%M:%S')
log_csv4['Complete Timestamp'] =  pd.to_datetime(log_csv4['Complete Timestamp'] , format='%m/%d/%Y %H:%M:%S')
parameters = {log_converter.Variants.TO_EVENT_LOG.value.Parameters.CASE_ID_KEY: 'Case ID'}
event_log4 = log_converter.apply(log_csv4, parameters=parameters, variant=log_converter.Variants.TO_EVENT_LOG)
event_log4 = sorting.sort_timestamp(event_log4,"time:timestamp", False)
event_log4

[{'attributes': {'concept:name': 'Case 178'}, 'events': [{'Case ID': 'Case 178', 'concept:name': 'Round Grinding - Machine 3', 'Resource': 'Machine 3 - Round Grinding', 'time:timestamp': Timestamp('2012-01-02 00:00:00'), 'Complete Timestamp': Timestamp('2012-01-02 04:50:00'), 'Span': '004:50', 'Work Order  Qty': 250, 'Part Desc.': 'Cable Head', 'Worker ID': 'ID4445', 'Report Type': 'D', 'Qty Completed': 31, 'Qty Rejected': 0, 'Qty for MRB': 0, 'Rework': nan, 'duration': '50:00.0'}, '..', {'Case ID': 'Case 178', 'concept:name': 'Round Grinding - Q.C.', 'Resource': 'Quality Check 1', 'time:timestamp': Timestamp('2012-03-14 00:00:00'), 'Complete Timestamp': Timestamp('2012-03-14 01:15:00'), 'Span': '000:00', 'Work Order  Qty': 250, 'Part Desc.': 'Cable Head', 'Worker ID': 'ID0937', 'Report Type': 'D', 'Qty Completed': 245, 'Qty Rejected': 0, 'Qty for MRB': 0, 'Rework': nan, 'duration': '15:00.0'}]}, '....', {'attributes': {'concept:name': 'Case 107'}, 'events': [{'Case ID': 'Case 107', 'c

In [3]:
activities_dict={}

for case_index, case in enumerate(event_log4):
        #print("\n case index: %d  case id: %s" % (case_index, case.attributes["concept:name"]))
        
        for event_index, event in enumerate(case):
            if(event["concept:name"] in activities_dict.keys()): #and activities_dict[event["concept:name"]] is not None):
                #print(activities_dict[event["concept:name"]])
                activities_dict[event["concept:name"]].append((case.attributes["concept:name"],event_index, event["concept:name"], event['Complete Timestamp']-event['time:timestamp']))
            else:
                activities_dict[event["concept:name"]]=[(case.attributes["concept:name"],event_index,event["concept:name"],event['Complete Timestamp']-event['time:timestamp'])]

#activities_dict
average_dict={}
for k,v in activities_dict.items():
    i=0
    for l in v:
        i+=1
        if(k in average_dict.keys()):
            average_dict[k]=[average_dict[k][0]+l[3].total_seconds(), i]
        else:
            average_dict[k]=[l[3].total_seconds(),i]

average_dict2={}
for k,v in average_dict.items():
    average_dict2[k]= v[0]/v[1]
activities_dict.keys()  
#print(activities_duration,i)

dict_keys(['Round Grinding - Machine 3', 'Round Grinding - Machine 2', 'Grinding Rework - Machine 27', 'Lapping - Machine 1', 'Grinding Rework - Machine 2', 'Fix - Machine 19', 'Turning & Milling Q.C.', 'Laser Marking - Machine 7', 'Round Grinding - Q.C.', 'Final Inspection Q.C.', 'Packing', 'Turning & Milling - Machine 4', 'Turning - Machine 8', 'Turning Q.C.', 'Milling - Machine 16', 'Round Grinding - Manual', 'Turning & Milling - Machine 9', 'Flat Grinding - Machine 11', 'Turning & Milling - Machine 8', 'Nitration Q.C.', 'Wire Cut - Machine 13', 'Turning & Milling - Machine 5', 'Grinding Rework', 'Turning & Milling - Machine 10', 'Round Grinding - Machine 12', 'Turning & Milling - Machine 6', 'Turn & Mill. & Screw Assem - Machine 9', 'Turning Rework - Machine 21', 'Flat Grinding - Machine 26', 'Round Grinding - Machine 19', 'Fix - Machine 3', 'Round Grinding - Machine 23', 'Milling - Machine 10', 'Fix - Machine 15', 'Grinding Rework - Machine 12', 'Stress Relief', 'Turning - Machine

In [4]:
cols=['Detected Weakness Row','Case ID','Weakness Type (AF/PA)','Weakness ID','Weakness Origin', 'Weakness Time','Weakness Information','Weakness Measurement', 'Weakness Level']
df=pd.DataFrame(columns=cols)
df

,Detected Weakness Row,Case ID,Weakness Type (AF/PA),Weakness ID,Weakness Origin,Weakness Time,Weakness Information,Weakness Measurement,Weakness Level


In [5]:
median_case_duration=case_statistics.get_median_caseduration (event_log4)
median_case_duration

1197480.0

In [6]:
case_description_durations=case_statistics.get_cases_description (event_log4)
for k,v in case_description_durations.items():
    row={cols[0]:k, cols[1]: k, cols[2]:'PA', cols[3]:'10',cols[4]:'Automatic detection',cols[5]:'',cols[6]:abs(v['caseDuration']-median_case_duration),cols[7]:'Distance from median case duration in seconds',cols[8]:'Case level'}
    df=df.append(row, ignore_index=True)

In [7]:
for case_index, case in enumerate(event_log4):
        #print("\n case index: %d  case id: %s" % (case_index, case.attributes["concept:name"]))
        
        for event_index, event in enumerate(case):
            row={cols[0]:case.attributes["concept:name"]+"-> Event "+str(event_index), cols[1]: case.attributes["concept:name"], cols[2]:'PA', cols[3]:'11',cols[4]:'Automatic detection',cols[5]:event['time:timestamp'],cols[6]: datetime.timedelta( abs(average_dict2[event["concept:name"]]-(event['Complete Timestamp']-event['time:timestamp']).total_seconds())),cols[7]:'Distance from average activity duration',cols[8]:'Activity level'}
            df=df.append(row, ignore_index=True)


In [8]:
df.to_excel("PA_dataframe.xlsx")

In [9]:
#df['Weakness Information'][0]


In [10]:
#df.to_excel("AF_dataframe.xlsx")

In [11]:


all_case_durations = case_statistics.get_all_casedurations(event_log4, parameters={
    case_statistics.Parameters.TIMESTAMP_KEY: "time:timestamp"})
         

In [12]:
#all_case_durations
z=case_statistics.get_cases_description(event_log4)
z['Case 12']['caseDuration']

1891320.0

In [13]:
median_case_duration = case_statistics.get_median_caseduration(event_log4, parameters={
    case_statistics.Parameters.TIMESTAMP_KEY: "time:timestamp"
})
median_case_duration

1197480.0

In [14]:
from pm4py.statistics.traces.log import case_arrival
case_arrival_ratio = case_arrival.get_case_arrival_avg(event_log4, parameters={
    case_arrival.Parameters.TIMESTAMP_KEY: "time:timestamp"})
case_arrival_ratio      

20070.0

In [15]:
from pm4py.statistics.performance_spectrum import algorithm as performance_spectrum
ps = performance_spectrum.apply(event_log4, ["Final Inspection Q.C.","Packing"], parameters={performance_spectrum.Parameters.ACTIVITY_KEY: "concept:name",
                                            performance_spectrum.Parameters.TIMESTAMP_KEY: "time:timestamp"})
ps

{'list_activities': ['Final Inspection Q.C.', 'Packing'],
 'points': [[1325671200.0, 1325721600.0],
  [1325683200.0, 1325721600.0],
  [1325690760.0, 1325808000.0],
  [1325750700.0, 1325808000.0],
  [1325757180.0, 1325808000.0],
  [1326006000.0, 1326240000.0],
  [1326031440.0, 1326067200.0],
  [1326183660.0, 1326240000.0],
  [1326207600.0, 1326326400.0],
  [1326277320.0, 1326672000.0],
  [1326283440.0, 1326412800.0],
  [1326288240.0, 1326585600.0],
  [1326356400.0, 1326585600.0],
  [1326445200.0, 1326585600.0],
  [1326447540.0, 1326585600.0],
  [1326456180.0, 1326585600.0],
  [1326457500.0, 1326585600.0],
  [1326610200.0, 1326672000.0],
  [1326611940.0, 1326758400.0],
  [1326619200.0, 1326672000.0],
  [1326629640.0, 1326758400.0],
  [1326630600.0, 1326844800.0],
  [1326805200.0, 1326844800.0],
  [1326873600.0, 1329091200.0],
  [1326882000.0, 1326931200.0],
  [1326956400.0, 1327190400.0],
  [1326967200.0, 1327017600.0],
  [1327215300.0, 1327276800.0],
  [1327217160.0, 1327276800.0],
  [1

In [16]:
from pm4py.objects.log.util import interval_lifecycle
enriched_log = interval_lifecycle.assign_lead_cycle_time(event_log4)
enriched_log

KeyError: 'lifecycle:transition'

In [ ]:
from pm4py.statistics.sojourn_time.log import get as soj_time_get

soj_time = soj_time_get.apply(event_log4, parameters={soj_time_get.Parameters.TIMESTAMP_KEY: "time:timestamp", soj_time_get.Parameters.START_TIMESTAMP_KEY: "start_timestamp"})
print(soj_time)